In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.linear_model import LinearRegression

# Load the Ames, Iowa housing dataset
data = pd.read_csv('housing_corr.csv')

# Split the data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define the predictor and response variables
X_train = train_data['TotalSF'].values.reshape(-1, 1)
y_train = train_data['SalePrice'].values
X_test = test_data['TotalSF'].values.reshape(-1, 1)
y_test = test_data['SalePrice'].values

# Define the linear regression model
model = LinearRegression()

# Define the cross-validation scheme
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# Calculate the R-squared score using cross-validation
scores = cross_val_score(model, X_test, y_test, cv=cv, scoring='r2')

# Print the average R-squared score across all cross-validation folds
print(f'Average R-squared score: {scores.mean()}')

Average R-squared score: 0.6804571491914195


In [43]:
# Split the data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define the predictor and response variables
X_train = train_data['OverallQual'].values.reshape(-1, 1)
y_train = train_data['SalePrice'].values
X_test = test_data['OverallQual'].values.reshape(-1, 1)
y_test = test_data['SalePrice'].values

# Define the linear regression model
model = LinearRegression()

# Define the cross-validation scheme
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# Calculate the R-squared score using cross-validation
scores = cross_val_score(model, X_test, y_test, cv=cv, scoring='r2')

# Print the average R-squared score across all cross-validation folds
print(f'Average R-squared score: {scores.mean()}')

Average R-squared score: 0.6005801779660601


In [44]:
# Extract the independent and dependent variables from both dataframes
X_train = train_df[['OverallQual', 'TotalSF']]
y_train = train_df['SalePrice'].values

X_test = test_df[['OverallQual', 'TotalSF']]
y_test = test_df['SalePrice'].values

# Create a LinearRegression model and fit it on the training data
model = LinearRegression()
model.fit(X_train, y_train)

# Define the cross-validation scheme
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# Perform cross-validation and calculate the R-squared score on the test data
scores = cross_val_score(model, X_test, y_test, scoring='r2', cv=cv)

# Print the mean R-squared score across all cross-validation folds
print('Mean R-squared score:', scores.mean())

Mean R-squared score: 0.7576260059571219


In [49]:
# Identify non-numeric features
non_numeric_columns = data.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

# Apply one-hot encoding to each non-numeric feature
data = pd.get_dummies(data, columns=non_numeric_columns)

# Preview the transformed dataset
print(data.head())

         PID  SalePrice  MSSubClass  LotFrontage  LotArea  OverallQual  \
0  909176150     126000          30          0.0     7890            6   
1  905476230     139500         120         42.0     4235            5   
2  911128020     124900          30         60.0     6060            5   
3  535377150     114000          70         80.0     8146            4   
4  534177230     227000          60         70.0     8400            8   

   OverallCond  MasVnrArea  ExterCond  BsmtQual  ...  \
0            6         0.0          2         3  ...   
1            5       149.0          2         4  ...   
2            9         0.0          2         3  ...   
3            8         0.0          3         2  ...   
4            6         0.0          2         4  ...   

   Neighborhood_st_Timber_WHITETAIL_LN  Neighborhood_st_Timber_WHITE_OAK_CIR  \
0                                    0                                     0   
1                                    0                    

In [51]:
# Split the data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define the predictor and response variables
X_train = train_data.drop('SalePrice', axis=1)
y_train = train_data['SalePrice'].values


X_test = test_data.drop('SalePrice', axis=1)
y_test = test_data['SalePrice'].values

# Define the linear regression model
ridge = Ridge(alpha=1.0)

# Define the cross-validation scheme
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# Calculate the R-squared score using cross-validation
scores = cross_val_score(ridge, X_test, y_test, cv=cv, scoring='r2')

# Print the average R-squared score across all cross-validation folds
print(f'Average R-squared score: {scores.mean()}')

/Users/mattwoolf/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/mattwoolf/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/mattwoolf/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/mattwoolf/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/mattwoolf/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instea

Average R-squared score: 0.8057430053026223


In [52]:
from sklearn.linear_model import Lasso

# Split the data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define the predictor and response variables
X_train = train_data.drop('SalePrice', axis=1)
y_train = train_data['SalePrice'].values


X_test = test_data.drop('SalePrice', axis=1)
y_test = test_data['SalePrice'].values

# Define the linear regression model
lasso = Lasso(alpha=1.0)

# Define the cross-validation scheme
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# Calculate the R-squared score using cross-validation
scores = cross_val_score(lasso, X_test, y_test, cv=cv, scoring='r2')

# Print the average R-squared score across all cross-validation folds
print(f'Average R-squared score: {scores.mean()}')

/Users/mattwoolf/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.412e+09, tolerance: 2.498e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mattwoolf/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.949e+09, tolerance: 2.561e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mattwoolf/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

Average R-squared score: 0.5998425889388569


/Users/mattwoolf/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.070e+09, tolerance: 2.637e+08
  model = cd_fast.enet_coordinate_descent(
/Users/mattwoolf/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.088e+10, tolerance: 2.581e+08
  model = cd_fast.enet_coordinate_descent(
